In [1]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.3 MB/s eta 0:00:00


In [5]:
%%writefile app.py

import streamlit as st
import pandas as pd
import math
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

df_35 = pd.read_csv("translations-3.5-turbo-processed.csv")
df4 = pd.read_csv("translations-4-processed.csv")
df_003 = pd.read_csv("translations-text-davinci-003-processed.csv")


def calculate_bias(df, option):
  selected_df = df[df["demographic"].str.contains(option)]
  num=0
  denom = 0
  for i,xs in selected_df.iterrows():
    if (xs[2] == 1 or xs[2] == 2):
      num += 1
    if (xs[2] == 0 or xs[2] == 2):
      denom += 1
  num +=1
  denom+=1
  bias = math.log(num)-math.log(denom)
  st.write(option + " bias: " + str(bias))


def calculate_bias_return(df, option):
  selected_df = df[df["demographic"].str.contains(option)]
  num=0
  denom = 0
  for i,xs in selected_df.iterrows():
    if (xs[2] == 1 or xs[2] == 2):
      num += 1
    if (xs[2] == 0 or xs[2] == 2):
      denom += 1
  num +=1
  denom+=1
  bias = math.log(num)-math.log(denom)
  return bias


def parameter_estimation(df):
  occupations = ["doctor", "nurse", "teacher", "accountant", "pastor", "security guard", "bartender", "engineer"]
  calibrations = []
  for occupation in occupations:
    calibrations.append(calculate_bias_return(df,occupation))

  mu = np.mean(calibrations)
  theta_squared = 0

  for calibration in calibrations:
    theta_squared += (calibration-mu)**2
  theta_squared = theta_squared/len(calibrations)
  return mu, theta_squared

def more_likely(mu, theta_squared,mu2, theta_squared2, C):
  X = norm(mu,math.sqrt(theta_squared))
  Y = norm(mu2,math.sqrt(theta_squared2))
  num = X.pdf(C)
  denom = Y.pdf(C)
  return float(num/denom)

st.header('Examining Gender Bias in Krio Translation by ChatGPT')
st.subheader('Calibration - Detecting Gender Bias')

option_model = st.selectbox('Choose a model.', ('<select>','gpt 3.5', 'gpt 4', 'davinci-003'))

if option_model != '<select>' and option_model=='gpt 3.5':
  option = st.selectbox('Choose an occupation.', ('<select>', "doctor", "nurse", "teacher", "accountant", "pastor", "security guard", "bartender", "engineer"))

  if option != '<select>':
    calculate_bias(df_35, option)

elif option_model=='gpt 4':
  option = st.selectbox('Choose a occupation.', ('<select>', "doctor", "nurse", "teacher", "accountant", "pastor", "security guard", "bartender", "engineer"))

  if option != '<select>':
    calculate_bias(df4, option)

elif option_model != '<select>' and option_model=='davinci-003':
  option = st.selectbox('Choose a occupation.', ('<select>', "doctor", "nurse", "teacher", "accountant", "pastor", "security guard", "bartender", "engineer"))

  if option != '<select>':
    calculate_bias(df_003, option)

st.subheader('Inference - Can We Tell Which GPT Did the Translations?')
option_model1 = st.selectbox('Choose a model.', ('<select>','gpt 3.5', 'gpt 4', 'davinci-003', 'davinci-002'), key="1")
option_model2 = st.selectbox('Choose a model.', ('<select>','gpt 3.5', 'gpt 4', 'davinci-003', 'davinci-002'), key="2")

if option_model1!="<select>" and option_model2!="<select>":
  model_to_df = {'gpt 3.5' : df_35, 'gpt 4' : df4, 'davinci-003' : df_003}
  mu, theta_squared = parameter_estimation(model_to_df[option_model1])
  mu2, theta_squared2 = parameter_estimation(model_to_df[option_model2])
  mean_calibration = st.text_input("Input mean bias")
  if (mean_calibration != ""):
    p = more_likely(mu,theta_squared,mu2,theta_squared2,float(mean_calibration))
    if (p > 1):
      st.write(option_model1 + " is more likely to be the translator")
    elif (p < 1):
      st.write(option_model2 + " is more likely to be the translator")
    else:
      st.write("no clue")

Overwriting app.py


In [6]:
!npm install localtunnel


up to date, audited 23 packages in 501ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [7]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

34.138.209.99


In [ ]:
!npx localtunnel --port 8501

your url is: https://three-flies-suffer.loca.lt
